# Lab 1A: Landing Zone Deployment

Deploy the central **Landing Zone (Hub)** that all project teams connect to.

## What Gets Deployed

| Resource | Purpose |
|----------|--------|
| AI Foundry Account | Central AI services hub |
| GPT-4.1-mini | Shared chat model for all teams |
| text-embedding-3-large | Shared embedding model for vector search |
| API Management (StandardV2) | Gateway with rate limiting for BYO Gateway |
| Storage Account | Shared storage |

## Step 1: Login to Azure

In [1]:
#!az login

## Step 2: Set Variables

In [2]:
RG = "foundry-lz-parent"
LOCATION = "eastus2"

## Step 3: Create Resource Group

In [3]:
!az group create -n "{RG}" -l "{LOCATION}" -o table

Location    Name
----------  -----------------
eastus2     foundry-lz-parent


## Step 4: Deploy Infrastructure

⏱️ Takes ~5-10 minutes (APIM is slow)

In [ ]:
import subprocess
PRINCIPAL_ID = subprocess.run('az ad signed-in-user show --query id -o tsv', shell=True, capture_output=True, text=True).stdout.strip()
!az deployment group create -g "{RG}" --template-file main.bicep -p deployerPrincipalId="{PRINCIPAL_ID}" -o table

## Step 5: Get Outputs

In [ ]:
import subprocess, json
from pathlib import Path

r = subprocess.run(f'az deployment group show -g "{RG}" -n main --query properties.outputs -o json', shell=True, capture_output=True, text=True)
out = json.loads(r.stdout)

ENDPOINT = out['aiEndpoint']['value']
APIM = out['apimUrl']['value']
APIM_NAME = out['apimName']['value']
APIM_SUB_NAME = out['apimSubscriptionName']['value']
MODEL = out['modelName']['value']
EMBEDDING_MODEL = out['embeddingModelName']['value']

# Get APIM subscription key
sub_id = subprocess.run('az account show --query id -o tsv', shell=True, capture_output=True, text=True).stdout.strip()
key_cmd = f'az rest --method POST --uri "https://management.azure.com/subscriptions/{sub_id}/resourceGroups/{RG}/providers/Microsoft.ApiManagement/service/{APIM_NAME}/subscriptions/{APIM_SUB_NAME}/listSecrets?api-version=2024-06-01-preview" --query primaryKey -o tsv'
APIM_KEY = subprocess.run(key_cmd, shell=True, capture_output=True, text=True).stdout.strip()

print(f"AI Endpoint:      {ENDPOINT}")
print(f"APIM URL:         {APIM}")
print(f"APIM Key:         {APIM_KEY[:8]}... (hidden)")
print(f"Chat Model:       {MODEL}")
print(f"Embedding Model:  {EMBEDDING_MODEL}")

# Write outputs to .env file in workspace root
env_file = Path("/workspaces/getting-started-with-foundry/.env")
env_file.write_text(f"""# Landing Zone Deployment Outputs
AI_ENDPOINT={ENDPOINT}
APIM_URL={APIM}
APIM_KEY={APIM_KEY}
MODEL_NAME={MODEL}
EMBEDDING_MODEL={EMBEDDING_MODEL}
""")
print(f"\n✅ Outputs saved to {env_file}")

## Step 6: Test the Chat Model

In [ ]:
!pip install openai azure-identity -q

In [7]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    azure_endpoint=ENDPOINT,
    azure_ad_token_provider=token_provider,
    api_version="2024-10-21"
)

response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "Say hello!"}]
)

print("✅ Working!")
print(response.choices[0].message.content)

✅ Working!
Hello! How can I assist you today?


## Step 7: Test the Embedding Model

In [8]:
import requests

# Test embedding model via APIM gateway
response = requests.post(
    f"{APIM}/deployments/{EMBEDDING_MODEL}/embeddings?api-version=2024-10-21",
    headers={"api-key": APIM_KEY, "Content-Type": "application/json"},
    json={"input": "Hello, space!", "model": EMBEDDING_MODEL}
)
response.raise_for_status()

embedding = response.json()["data"][0]["embedding"]
print(f"✅ Embedding model working!")
print(f"   Model: {EMBEDDING_MODEL}")
print(f"   Dimensions: {len(embedding)}")
print(f"   Sample: [{embedding[0]:.6f}, {embedding[1]:.6f}, {embedding[2]:.6f}, ...]")

✅ Embedding model working!
   Model: text-embedding-3-large
   Dimensions: 3072
   Sample: [-0.029543, -0.029624, -0.016452, ...]


## Done!

Landing Zone deployed. Next: **Lab 1B** to connect project teams.

---

## Cleanup (Optional)

In [9]:
#!az group delete -n "{RG}" --yes --no-wait